In [1]:
from TTS.utils.manage import ModelManager

import sys
sys.path.insert(0, "../Src/")
import generateVoices as gv
from os.path import exists
import pandas as pd
from pydub import AudioSegment
import speech_recognition as sr
import jiwer
import json
import numpy as np

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
def evaluateWER(generatedAudioFileName, sentence):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(generatedAudioFileName) as source:
            audio = recognizer.record(source)
            hypothesis = recognizer.recognize_google(audio)
        wer = jiwer.wer(sentence, hypothesis)
        print("WER:", wer)
        return wer
    except sr.UnknownValueError:
        print(f"Could not understand audio: {generatedAudioFileName}")
        return 1.0
    except Exception as e:
        print(f"Error processing {generatedAudioFileName}: {e}")
        return 1.0

In [ ]:
def evaluateWERForModel(model_dir_name, outputCSVFile):
    outputDF = pd.read_csv(outputCSVFile)
    speakers = outputDF['speakerId']
    wordErrorRateArray = []
    for speaker in speakers:
        sentence = outputDF[outputDF['speakerId'] == speaker]['generatedSentence'].values[0]
        generatedAudioFileName = f'../Data/ttsOutputs/{model_dir_name}/{speaker}.wav'
        #evaluate word error rate for geneatedAudioFileName vs sentence
        wer = evaluateWER(generatedAudioFileName, sentence)
        wordErrorRateArray.append(wer)
    if not wordErrorRateArray:
        print(f"No valid WER values for model {model_dir_name}")
        return 1.0
    np.mean(wordErrorRateArray), np.std(wordErrorRateArray, ddof=1) / np.sqrt(len(wordErrorRateArray))


In [ ]:
def main():
    wordErrorRateResults = []
    if exists('../Data/wordErrorRateResults.json'):
        with open('../Data/wordErrorRateResults.json', 'r') as f:
            wordErrorRateResults = json.load(f) 
    
    englishModels = gv.getRawEnglishModelNames()
    for model in englishModels:
        print(f"Processing model: {model}")
        model_dir_name = model.replace("/", "_")
        outputCSVFile = f'../Data/ttsOutputs/{model_dir_name}_generatedSentences.csv'
        if not exists(outputCSVFile):
            continue
        averageWER, stdErrorWER = evaluateWERForModel(model_dir_name, outputCSVFile)
        wordErrorRateResults.append({
            "model": model,
            "averageWER": averageWER,
            "stdErrorWER": stdErrorWER
        })
        with open('../Data/wordErrorRateResults.json', 'w') as f:
            json.dump(wordErrorRateResults, f)
    wordErrorRateResults = pd.DataFrame(wordErrorRateResults)
    wordErrorRateResults = wordErrorRateResults.sort_values(by=['averageWER'], ascending=False)
    display(wordErrorRateResults)
    wordErrorRateResults.to_csv('../Data/wordErrorRateResults.csv', index=False)
    print('done')

In [5]:
if __name__ == '__main__':
    main()


 Name format: type/language/dataset/model
 1: tts_models/multilingual/multi-dataset/xtts_v2
 2: tts_models/multilingual/multi-dataset/xtts_v1.1
 3: tts_models/multilingual/multi-dataset/your_tts
 4: tts_models/multilingual/multi-dataset/bark
 5: tts_models/bg/cv/vits
 6: tts_models/cs/cv/vits
 7: tts_models/da/cv/vits
 8: tts_models/et/cv/vits
 9: tts_models/ga/cv/vits
 10: tts_models/en/ek1/tacotron2 [already downloaded]
 11: tts_models/en/ljspeech/tacotron2-DDC [already downloaded]
 12: tts_models/en/ljspeech/tacotron2-DDC_ph [already downloaded]
 13: tts_models/en/ljspeech/glow-tts [already downloaded]
 14: tts_models/en/ljspeech/speedy-speech [already downloaded]
 15: tts_models/en/ljspeech/tacotron2-DCA [already downloaded]
 16: tts_models/en/ljspeech/vits [already downloaded]
 17: tts_models/en/ljspeech/vits--neon [already downloaded]
 18: tts_models/en/ljspeech/fast_pitch [already downloaded]
 19: tts_models/en/ljspeech/overflow [already downloaded]
 20: tts_models/en/ljspeech/

,model,averageWER
1,tts_models/en/ljspeech/tacotron2-DDC,0.988822
0,tts_models/en/ek1/tacotron2,0.521099


done
